In [ ]:
#Test X: All Zero Solution

#ABSTRACT ------------------------------------------------------------------------------------------------------------
- We need to make sure the solver does not think that an all zero solution (as in no routes used, 
no customers visited) is a viable solution.

#METHOD ---------------------------------------------------------------------------------------------------------------
- Using different samples of data, we will find the cost when all variables are zero. Then compare the
cost with the lowest cost sample. Where the all zero solution must be notably larger. Below are the 
different sets of data used:
    
- Data set 1:
nodes = [0, 1, 2]
customers = [1, 2]
vehicles = ['a', 'b']

t = {
    (0,1): 3, (1,0): 3,
    (0,2): 5, (2,0): 5,
    (1,2): 4, (2,1): 4
}

- Data set 2:
nodes = [0, 1, 2]
customers = [1, 2]
vehicles = ['a', 'b']

t = {
    (0,1): 2, (1,0): 2,
    (0,2): 7, (2,0): 7,
    (1,2): 3, (2,1): 3
}

- Data set 3:
nodes = [0, 1, 2]
customers = [1, 2]
vehicles = ['a', 'b']

t = {
    (0,1): 4, (1,0): 4,
    (0,2): 6, (2,0): 6,
    (1,2): 2, (2,1): 2
}

# ADDED CODE ------------------------------------------------------------------------------------------------------------
ZeroSample = {v: 0 for v in bqm.variables}
CostZero = bqm.energy(ZeroSample)
print("All zero BQM cost:", CostZero)

solver = dimod.ExactSolver()
result = solver.sample(bqm)
print(result)

# RESULTS --------------------------------------------------------------------------------------------------------------
- Data set 1:
All Zero cost: 300   ExactSolver lowest cost: 16
- Data set 2:
All Zero cost: 420   ExactSolver lowest cost: 18
- Data set 3:
All Zero cost: 360   ExactSolver lowest cost: 20
    
# CONCLUSION ------------------------------------------------------------------------------------------------------------
- The test results came out as expected. Hence the algorithm correctly penalizes an all zero solution.

In [9]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

nodes = [0, 1, 2]
customers = [1, 2]
vehicles = ['a', 'b']

t = {
    (0,1): 4, (1,0): 4,
    (0,2): 6, (2,0): 6,
    (1,2): 2, (2,1): 2
}

# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"Visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(
        quicksum(x[0,j,k] for j in customers) == 1,
        label=f"Start_{k}"
    )
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) == 1,
        label=f"End_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )


# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)

#---TEST-------------------------------------------------
ZeroSample = {v: 0 for v in bqm.variables}
CostZero = bqm.energy(ZeroSample)
print("All zero BQM cost:", CostZero)

solver = dimod.ExactSolver()
result = solver.sample(bqm)
print(result)
#--------------------------------------------------------

All zero BQM cost: 360.0
     x_0_1_a x_0_1_b x_0_2_a x_0_2_b x_1_0_a x_1_0_b ... x_2_1_b energy num_oc.
452        0       1       1       0       0       1 ...       0   20.0       1
1006       1       0       0       1       1       0 ...       0   20.0       1
445        1       1       0       0       0       1 ...       0   80.0       1
797        1       1       0       0       1       0 ...       0   80.0       1
2011       0       1       1       0       1       1 ...       0   80.0       1
4049       1       0       0       1       1       1 ...       1   80.0       1
625        1       0       0       1       0       0 ...       0   84.0       1
763        0       1       1       0       0       0 ...       0   84.0       1
1047       0       0       1       1       1       0 ...       0   84.0       1
3639       0       0       1       1       0       1 ...       1   84.0       1
385        1       0       0       0       0       0 ...       0  132.0       1
771        0   